In [1]:
#Compute limits
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 19 17:14:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers
!pip install accelerate
!pip install datasets
#!pip install rouge_score
!pip install peft
!pip install trl
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 15.1 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import statistics

In [21]:
#dftr = pd.read_csv('/content/stories_csv_bs.csv')
dftr = pd.read_excel('/content/stories_excel_bs.xlsx')
dftr

,Prompt,Title,Part,Story
0,గాడిద జోస్యం అనే చందమామ కథ చెప్పు?,గాడిద జ్యోస్యం,మొదటి భాగం.,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...
1,NaN,గాడిద జ్యోస్యం,రెండవ భాగం.,ఆ సమయానికి ఊరి చాకలి ఉతికిన బట్టలు గాడిద పైన వ...
2,NaN,గాడిద జ్యోస్యం,మూడవ భాగం.,"సిద్ధాంతి గారికి ఏమి చేయాలో తోచలేదు, చాకలి చెప..."
3,ఓవర్ కాన్ఫిడెన్స్ ఉండకూడదు అనే దానికి ఉదాహరణగ...,గాడిద జ్యోస్యం,మొదటి భాగం.,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...
4,NaN,గాడిద జ్యోస్యం,రెండవ భాగం.,ఆ సమయానికి ఊరి చాకలి ఉతికిన బట్టలు గాడిద పైన వ...
...,...,...,...,...
105,NaN,చౌక బేరం,NaN,\n\nహంసవరం అనే ఊరిలో వరాలయ్య అనే ఓ మధ్య తరగతి ...
106,NaN,నేను పిసినారినా?,NaN,\n\nపార్వతి పుట్టింటిలో కొద్ది రోజులు గడిపిన త...
107,NaN,రాజార్హత,NaN,\n\nఅవంతి సింహాసనం ఖాళీ అయింది. రాజు వీరమల్లుడ...
108,NaN,రుజువు,NaN,\n\n\n\nసుబ్బరాజుదికొబ్బరికాయల వ్యాపారం. ఆ ఊర్...


In [6]:
import os
import transformers
import torch
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

# Download Gemma 2b-it base model
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
# Configure LoRA
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM"
)

In [22]:
print("Max sequence length:", tokenizer.model_max_length)
dftr['Story_len'] = dftr['Story'].apply(lambda x: len(tokenizer(x)['input_ids']))
dftr['Story_len'].describe()

Max sequence length: 1000000000000000019884624838656


,Story_len
count,110.000000
mean,1917.663636
std,796.339012
min,547.000000
25%,1340.250000
50%,1889.500000
75%,2581.500000
max,3487.000000


In [17]:
dftr['Story_len'][:16]

,Story_len
0,547
1,761
2,614
3,547
4,761
5,614
6,584
7,828
8,584
9,828


In [27]:
dftr[:16]

,Prompt,Title,Part,Story,Story_len
0,గాడిద జోస్యం అనే చందమామ కథ చెప్పు?,గాడిద జ్యోస్యం,మొదటి భాగం.,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...,547
1,NaN,గాడిద జ్యోస్యం,రెండవ భాగం.,ఆ సమయానికి ఊరి చాకలి ఉతికిన బట్టలు గాడిద పైన వ...,761
2,NaN,గాడిద జ్యోస్యం,మూడవ భాగం.,"సిద్ధాంతి గారికి ఏమి చేయాలో తోచలేదు, చాకలి చెప...",614
3,ఓవర్ కాన్ఫిడెన్స్ ఉండకూడదు అనే దానికి ఉదాహరణగ...,గాడిద జ్యోస్యం,మొదటి భాగం.,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...,547
4,NaN,గాడిద జ్యోస్యం,రెండవ భాగం.,ఆ సమయానికి ఊరి చాకలి ఉతికిన బట్టలు గాడిద పైన వ...,761
5,NaN,గాడిద జ్యోస్యం,మూడవ భాగం.,"సిద్ధాంతి గారికి ఏమి చేయాలో తోచలేదు, చాకలి చెప...",614
6,బ్రహ్మప్రళయం గురించి ఒక చందమామ కథ చెప్పు?,బ్రహ్మ ప్రళయం,మొదటి భాగం.,"బ్రహ్మదేవుడు భూమిపై మానవులను సృష్టించి, నాయనలా...",584
7,NaN,బ్రహ్మ ప్రళయం,రెండవ భాగం.,తరువాత బ్రహ్మ నియమించిన వారిలో మొదటి వాడు బ్రహ...,828
8,ఏదైనా మంచి చందమామ కథ చెప్పు?,బ్రహ్మ ప్రళయం,మొదటి భాగం.,"బ్రహ్మదేవుడు భూమిపై మానవులను సృష్టించి, నాయనలా...",584
9,NaN,బ్రహ్మ ప్రళయం,రెండవ భాగం.,తరువాత బ్రహ్మ నియమించిన వారిలో మొదటి వాడు బ్రహ...,828


In [31]:
import json
train_data = []
for index, row in dftr[:16].iterrows():
  train_data += [{'prompt': row.iloc[0], 'title' : row.iloc[1], 'part' : row.iloc[2], 'story' : row.iloc[3] }]
with open('train_file.json', 'w') as file:
    json.dump(train_data, file)

In [10]:
# Tokenize the Dataset - 1

from datasets import load_dataset
dataset = load_dataset('json', data_files = {'train':'train_file.json'})
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples, chunk_size=1024):
    #inputs = examples['prompt'] + examples['title'] + examples['story'] + tokenizer.eos_token
    inputs = ["Prompt: " + p + "\nTitle:" + t + '\Story:'+ s
              for p, t, s in zip(examples['prompt'], examples['title'], examples['story'])]
    chunks = []
    for input in inputs:
      model_inputs = tokenizer(input, truncation=False)
      i = 0
      while i < len(model_inputs["input_ids"]):
        if i == 0:
          chunk = {key: value[i : i + chunk_size] for key, value in model_inputs.items()}
          i = i + chunk_size
        else:
          chunk = {key: value[i - 128 : i + chunk_size - 128] for key, value in model_inputs.items()}
          i = i + chunk_size - 128
        for key in chunk:
              padding_length = chunk_size - len(chunk[key])
              if padding_length > 0:
                  chunk[key] = chunk[key] + [tokenizer.pad_token_id] * padding_length
        chunk['labels'] = chunk['input_ids'].copy()
        chunks.append(chunk)

    chunked_dict = {key: [chunk[key] for chunk in chunks] for key in chunks[0]}
    return chunked_dict

tokenized_dataset = dataset.map(preprocess_function, remove_columns=dataset['train'].column_names, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [32]:
# Tokenize the Dataset - 2

from datasets import load_dataset
dataset = load_dataset('json', data_files = {'train':'train_file.json'})
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):

    inputs = [
    (str(pro) if pro is not None else "") + "\n" +
    (str(t) if t is not None else "") + " " +
    (str(par) if par is not None else "") + "\n" +
    (str(s) if s is not None else "") +
    tokenizer.eos_token
    for pro, t, par, s in zip(examples['prompt'], examples['title'], examples['part'], examples['story'])
    ]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = model_inputs['input_ids'].copy()
    model_inputs['labels'] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function,  batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [11]:
print(tokenizer.encode('మొదటి భాగం..'))
print(tokenizer.encode('రెండవ భాగం..'))
print(tokenizer.encode('మూడవ భాగం..'))
print(tokenizer.encode('నాలుగవ భాగం..'))
print(tokenizer.encode('ఐదవ భాగం..'))
print(tokenizer.encode('ఆరవ భాగం..'))
print(tokenizer.encode('ఏడవ భాగం..'))
print(tokenizer.encode('కథ సమాప్తం..'))

[2, 236977, 239594, 237231, 90647, 118231, 236356, 237467, 236592, 723]
[2, 236442, 238148, 71767, 236936, 118231, 236356, 237467, 236592, 723]
[2, 195209, 237573, 236936, 118231, 236356, 237467, 236592, 723]
[2, 236498, 81574, 237467, 236936, 118231, 236356, 237467, 236592, 723]
[2, 245275, 237231, 236936, 118231, 236356, 237467, 236592, 723]
[2, 239647, 236442, 236936, 118231, 236356, 237467, 236592, 723]
[2, 241913, 237573, 236936, 118231, 236356, 237467, 236592, 723]
[2, 236698, 240757, 38809, 130128, 112930, 236930, 236592, 723]


In [33]:
for i in range(len(tokenized_dataset['train'])):
  print(tokenizer.decode((tokenized_dataset['train'][i]['input_ids'])))


<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>

In [24]:
import gc
torch.cuda.empty_cache()
del dftr
gc.collect()

155

In [36]:
from transformers import Trainer, TrainingArguments

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Training arguments configuration
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_strategy='epoch',
    save_strategy = 'epoch',
    report_to="none",
    fp16=True,
    optim="adamw_torch"
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    #eval_dataset=tokenized_dataset,  # Ideally, separate eval dataset
)

trainer.train()

Step,Training Loss
16,2.152800
32,1.486500
48,0.902800
64,0.421200
80,0.131600
96,0.038500
112,0.039700
128,0.024500
144,0.013700
160,0.012100


TrainOutput(global_step=160, training_loss=0.5223288061097264, metrics={'train_runtime': 189.3405, 'train_samples_per_second': 0.845, 'train_steps_per_second': 0.845, 'total_flos': 1957911173529600.0, 'train_loss': 0.5223288061097264, 'epoch': 10.0})

In [1]:
# LOADING THE SAVED MODEL
!unzip -q '/content/gemma2_2b_trained.zip' -d '/content/gemma2_2b_trained'

In [2]:
# LOADING THE SAVED MODEL
from transformers import AutoTokenizer, AutoModelForCausalLM

# Reload the model and tokenizer
model_path = "/content/gemma2_2b_trained"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

print("Model and tokenizer successfully loaded.")

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model and tokenizer successfully loaded.


In [3]:
import torch
model = model.to(torch.device('cuda'))
model.device

device(type='cuda', index=0)

In [45]:
model.eval()

max_steps = 400  # Total desired length of tokens

input_ids = tokenizer.encode("గాడిద జోస్యం అనే చందమామ కథ చెప్పు?", return_tensors='pt').to('cuda')
attention_mask = torch.ones(input_ids.shape).to('cuda')
#output = model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=1024)
#gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
#print(gen_text)

generated_story = ""
for _ in range(max_steps):
    output = model.generate(input_ids, return_dict_in_generate=True, max_new_tokens=1)
    next_token = tokenizer.decode(output.sequences[:, -1])
    generated_story += next_token
    if "కథ సమాప్తం." in next_token or generated_story.endswith("కథ సమాప్తం."):
        break
print(generated_story)

In [46]:
input_ids

tensor([[     2,  72245, 107341, 237231,  96206, 237567, 237093,  47252, 236592,
          50676, 179479,  58048,  51660, 236977, 116014,  38878, 240757, 219126,
         112930, 115117, 235336]], device='cuda:0')

In [23]:
tokenizer.decode(output[0], skip_special_tokens=True)

NameError: name 'output' is not defined

In [ ]:
story = "Prompt: " + "స్నేహితుల మధ్య వ్యత్యాసం గురించి ఒక కథ చెప్పు?"
input_ids = tokenizer(story, return_tensors="pt").input_ids

# Generate long text iteratively
max_story_length = 3000  # Total desired length of tokens
generated_ids = input_ids

while generated_ids.shape[1] < max_story_length:
    output_ids = model.generate(
        input_ids=generated_ids,
        max_length=generated_ids.shape[1] + 1024,  # Generate additional tokens
        temperature=0.8,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    # Append newly generated tokens
    generated_ids = output_ids

# Decode final output
final_story = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated Story:")
print(final_story)